---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [2]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [3]:
def answer_one():
    with open('Employee_Movie_Choices.txt') as file:
        data = file.readlines()
    choices_array = []
    for line in data:
        choices_array.append([line.split()[0], " ".join(line.split()[1:])])
    choices_df = pd.DataFrame(choices_array, columns = ['Employee', 'Movie']).drop(0)

    G5 = nx.from_pandas_dataframe(choices_df, 'Employee', 'Movie', edge_attr = None)
    return(G5)

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [4]:
def answer_two():
    B = nx.Graph()
    nodes = bipartite.sets(answer_one())
    B.add_nodes_from(list(nodes[0]), type = 'employee')
    B.add_nodes_from(list(nodes[1]), type = 'movie')
    B.add_edges_from(answer_one().edges())
    return(B)

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [44]:
def answer_three():
    nodes = bipartite.sets(answer_two())
    employee_names = nodes[0]
    graph = bipartite.weighted_projected_graph(answer_two(), employees)
    graph.edges(data = True)
    #plot_graph(graph, weight_name = 'weight')
    return(graph)

In [48]:
#answer_three().nodes(data = True)

In [47]:
#answer_two().edges()

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [36]:
def answer_four():
    import numpy as np
    common_movie_adjacency = nx.to_pandas_dataframe(answer_three())
    common_movie_adjacency.values[[np.arange(len(common_movie_adjacency))]*2] = np.nan
    common_edgelist = common_movie_adjacency.stack().reset_index()
    common_edgelist.columns = ['Level0','Level1','Connection']
    common_edgelist = common_edgelist.sort_values(['Level0','Level1']).set_index(['Level0','Level1'])
    employee_relationship =  pd.read_csv('Employee_Relationships.txt', delimiter='\t', header = None)
    employee_relationship.columns = ['Level0','Level1','Score']
    employee_relationship.set_index(['Level0','Level1'], inplace = True)
    connection = employee_relationship.join(common_edgelist, how = 'left')
    return(connection.corr().iloc[0].iloc[1])